In [2]:
import pandas as pd

In [3]:
schema = pd.read_csv('stats19_schema.csv')
coll_schema = schema[schema.table=='Accident']
cas_schema = schema[schema.table=='Casualty']

In [4]:
coll_url = r"https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-collision-last-5-years.csv"
cas_url = r"https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-casualty-last-5-years.csv"

coll_df = pd.read_csv(coll_url, low_memory = False)
cas_df = pd.read_csv(cas_url, low_memory = False)

In [5]:
coll_df = coll_df[coll_df.police_force == 14]

In [6]:
coll_df = pd.concat([coll_df[name].map(coll_schema[coll_schema.variable == name].set_index('code').label).copy() if name in coll_schema.variable.unique() else coll_df[name] for name in coll_df.columns], axis=1)

In [7]:
cas_df = pd.concat([
    cas_df[name].map(cas_schema[cas_schema.variable == name].set_index('code').label).copy() if name in cas_schema.variable.unique() else cas_df[name] for name in cas_df.columns], axis=1)

In [8]:
cas_df.accident_index.isin(coll_df.accident_index).value_counts()

False    678417
True      14611
Name: accident_index, dtype: int64

In [9]:
joined = coll_df.merge(cas_df, on='accident_index', how='left')

In [11]:
joined.to_csv('dft_statistics_collision_and_casualty_last_5_years.csv', index=False)